In [6]:
%pip install pandas
%pip install statsmodels
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.

  Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import warnings
from sklearn.metrics import mean_absolute_error
import logging
import os 

In [37]:
file_path = r"C:\temp\All_products_24_bcp.csv"
churn_df = pd.read_csv(file_path, encoding="utf-8")


print(churn_df.head())

   MANAD_1DAG   ATL  ATLD   LAF   LAM   LBR
0  2019-01-01  0.04  0.07  0.09  0.05  0.06
1  2019-02-01  0.06  0.09  0.08  0.02  0.04
2  2019-03-01  0.04  0.10  0.08  0.05  0.04
3  2019-04-01  0.03  0.08  0.05  0.03  0.05
4  2019-05-01  0.04  0.09  0.09  0.03  0.06


In [42]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import logging
import warnings
from datetime import datetime

# Set up logging
logging.basicConfig(filename='model_warnings.log', level=logging.WARNING)

# Load the data
file_path = r"C:\temp\All_products_24_bcp.csv"
churn_df = pd.read_csv(file_path, encoding="utf-8")

# Convert the 'MANAD_1DAG' column to datetime and set it as index
churn_df['MANAD_1DAG'] = pd.to_datetime(churn_df['MANAD_1DAG'])
churn_df.set_index('MANAD_1DAG', inplace=True)
churn_df.sort_index(inplace=True)

# Function to check for NaN values
def check_nan(data, step):
    nan_count = np.isnan(data).sum()
    if nan_count > 0:
        print(f"NaN values found at step {step}: {nan_count}")
    return nan_count == 0

# Function to extract trend using moving average
def extract_trend(series, window=12):
    trend = series.rolling(window=window, center=True).mean()
    trend = trend.fillna(method='ffill').fillna(method='bfill')
    return trend

# List to hold DataFrames of forecasts for each product
all_dfs = []
evaluation_metrics = []

# Get the last date in the dataset
last_date = churn_df.index[-1]

# Calculate the number of months to forecast
months_to_forecast = 12 + (12 - last_date.month)

# Loop through each product column
for product in churn_df.columns:
    try:
        print(f"\nProcessing product: {product}")
        
        series = churn_df[product].dropna()
        
        print(f"Series length for {product}: {len(series)}")
        print(f"Series range: {series.min():.4f} to {series.max():.4f}")
        
        if len(series) < 24:
            print(f"Skipping {product} due to insufficient data")
            continue

        if not check_nan(series.values, "Initial series"):
            continue

        # Apply log transformation
        series_log = np.log(series.clip(lower=1e-8))
        
        if not check_nan(series_log.values, "After log transformation"):
            continue

        # Extract trend using moving average
        trend = extract_trend(series_log)
        residual = series_log - trend

        if not check_nan(trend.values, "Trend") or not check_nan(residual.values, "Residual"):
            continue

        # Set up the SARIMAX model
        model = SARIMAX(residual, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), 
                        enforce_stationarity=False, enforce_invertibility=False)
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            results = model.fit(disp=False, maxiter=1000)

        # Forecast future churn
        forecast = results.get_forecast(steps=months_to_forecast)
        mean_forecast = forecast.predicted_mean

        if not check_nan(mean_forecast.values, "Mean forecast"):
            continue

        # Add back trend component to the forecasted residuals 
        trend_forecast = trend.iloc[-1] + np.arange(1, months_to_forecast + 1) * (trend.iloc[-1] - trend.iloc[-13]) / 12
        mean_forecast += trend_forecast

        if not check_nan(mean_forecast.values, "Mean forecast after adding trend"):
            mean_forecast = mean_forecast.fillna(method='ffill').fillna(method='bfill')
            print("Filled NaN values in mean forecast")

        # Reverse the log transformation and handle negative predictions
        mean_forecast = np.exp(mean_forecast).clip(lower=0)

        print("Mean forecast shape (final):", mean_forecast.shape)

        # Create a date range for the forecast
        forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=months_to_forecast, freq='MS')

        # Create a DataFrame for the forecast
        forecast_df = pd.DataFrame({
            f'Forecasted_{product}': mean_forecast
        }, index=forecast_dates)

        all_dfs.append(forecast_df)

        # Model Evaluation
        # Use the last 12 months of actual data for evaluation
        actual_last_year = series[-12:]
        forecast_last_year = results.get_forecast(steps=12)
        forecast_mean_last_year = np.exp(forecast_last_year.predicted_mean + trend[-12:]).clip(lower=0)

        mae = mean_absolute_error(actual_last_year, forecast_mean_last_year)
        rmse = np.sqrt(mean_squared_error(actual_last_year, forecast_mean_last_year))
        mape = np.mean(np.abs((actual_last_year - forecast_mean_last_year) / actual_last_year)) * 100

        evaluation_metrics.append({
            'Product': product,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
        })

        print(f"Successfully processed {product}")
    except Exception as e:
        print(f"Error processing {product}: {str(e)}")
        logging.warning(f"Model fitting failed for {product}: {e}")

print(f"\nNumber of successfully processed products: {len(all_dfs)}")

# Concatenate all the individual DataFrames horizontally
if len(all_dfs) > 0:
    final_df = pd.concat(all_dfs, axis=1)
    
    # Save the forecasts to a CSV file
    final_df.to_csv("product_forecasts_full_year.csv")
    
    print("Final DataFrame:")
    print(final_df)

    # Create and save evaluation metrics
    eval_df = pd.DataFrame(evaluation_metrics)
    eval_df.to_csv("model_evaluation_metrics.csv", index=False)
    print("\nEvaluation Metrics:")
    print(eval_df)
else:
    print("No products were successfully processed. Check the data and error messages above.")



Processing product: ATL
Series length for ATL: 67
Series range: 0.0100 to 0.0600


C:\Users\jonatan.oldenburg\AppData\Local\Temp\ipykernel_107668\1152261005.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trend = trend.fillna(method='ffill').fillna(method='bfill')
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean forecast shape (final): (17,)
Error processing ATL: Found input variables with inconsistent numbers of samples: [12, 24]

Processing product: ATLD
Series length for ATLD: 67
Series range: 0.0500 to 0.1400


C:\Users\jonatan.oldenburg\AppData\Local\Temp\ipykernel_107668\1152261005.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trend = trend.fillna(method='ffill').fillna(method='bfill')
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean forecast shape (final): (17,)
Error processing ATLD: Found input variables with inconsistent numbers of samples: [12, 24]

Processing product: LAF
Series length for LAF: 67
Series range: 0.0300 to 0.1100


C:\Users\jonatan.oldenburg\AppData\Local\Temp\ipykernel_107668\1152261005.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trend = trend.fillna(method='ffill').fillna(method='bfill')
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean forecast shape (final): (17,)
Error processing LAF: Found input variables with inconsistent numbers of samples: [12, 24]

Processing product: LAM
Series length for LAM: 67
Series range: 0.0000 to 0.0500
Mean forecast shape (final): (17,)
Error processing LAM: Found input variables with inconsistent numbers of samples: [12, 24]

Processing product: LBR
Series length for LBR: 67
Series range: 0.0100 to 0.0800


C:\Users\jonatan.oldenburg\AppData\Local\Temp\ipykernel_107668\1152261005.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trend = trend.fillna(method='ffill').fillna(method='bfill')
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jonatan.oldenburg\OneDrive - Solita Oy\Desktop\LRFM\Churnprognos\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\jonatan.oldenburg\AppData\Local\Temp\ipykernel_107668\1152261005.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill

Mean forecast shape (final): (17,)
Error processing LBR: Found input variables with inconsistent numbers of samples: [12, 24]

Number of successfully processed products: 5
Final DataFrame:
            Forecasted_ATL  Forecasted_ATLD  Forecasted_LAF  Forecasted_LAM  \
2024-08-01        0.019932         0.045932        0.073373        0.000288   
2024-09-01        0.013701         0.049990        0.058230        0.029325   
2024-10-01        0.019904         0.052776        0.052952        0.037184   
2024-11-01        0.024718         0.054367        0.047929        0.023372   
2024-12-01        0.019527         0.051249        0.041905        0.035286   
2025-01-01        0.023859         0.051277        0.055981        0.054071   
2025-02-01        0.029708         0.059150        0.064291        0.034817   
2025-03-01        0.029737         0.054920        0.098910        0.040515   
2025-04-01        0.030613         0.051070        0.084935        0.040210   
2025-05-01        0.0